In [3]:
##### DOWNLOAD DATASET #####
from IPython.display import clear_output
from google.colab import files
files.upload()
!ls -lha kaggle.json
!pip install -q kaggle # installing the kaggle package
!mkdir -p ~/.kaggle # creating .kaggle folder where the key should be placed
!cp kaggle.json ~/.kaggle/ # move the key to the folder
!pwd # checking the present working directory
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d gsimonx37/letterboxd
!unzip /content/letterboxd.zip -d /content/letterbox/
clear_output()
############################

In [4]:
########### SPARK CONTEXT #####################
import pandas as pd
import itertools
from tqdm import tqdm

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better

sc = spark.sparkContext
###############################################

In [50]:
############## BASKET CREATION ################
#TODO do it using SPARK directly -> cars = spark.read.csv('cars.csv', header=True, sep=";")
actors = pd.read_csv("letterbox/actors.csv")
actors = actors
baskets = actors.groupby("id")["name"].apply(list)
#baskets = baskets.sample(10000)

print("number of baskets: " + str(len(baskets)))
print("biggest basket: " + str(baskets.map(len).max()))
print(baskets)
baskets_RDD = sc.parallelize(baskets).cache()
##############################################

number of baskets: 603163
biggest basket: 451
id
1000001    [Margot Robbie, Lewis Easter, Onyemachi Ejimof...
1000002    [Rosie Peralta, Anna Elisabeth Rihlmann, Andre...
1000003    [Randall Archer, Boon Pin Koh, Efka Kvaracieju...
1000004    [Leonard Termo, Greg Bronson, Michael Arturo, ...
1000005    [Lena Georgas, Jeff Hephner, Elyes Gabel, Broo...
                                 ...                        
1896377    [Marat Basharov, Sergey Rost, Gleb Kalyuzhny, ...
1896382                                   [Marine Petrosyan]
1896387                                        [Rebecca Jim]
1896391                                       [Ko Shibasaki]
1896393                                    [Atsuhiro Inukai]
Name: name, Length: 603163, dtype: object


In [54]:
test_basket = [
    ['a','b','c','d','e','f','g','h','i','l'],
    ['a','b','c','d','e','f',],
    ['a','b','c','d','e','f','q','p','o','n'],
    ['a','b','c','d','e','f','r','s','t','u'],
    ['a','b','c'],
    ['f','q'],
    ['f','q','z'],
    ['f','u','q'],
    ['n','o'],
    ['o','n'],
    ['n','o','i']
]
test_RDD = sc.parallelize(test_basket).cache()

In [91]:
## APRIORI
def apriori(chunk, s, tot_bsk):

  n_bsk = 0 # number of baskets in chunk used to calculate support
#  trsl = dict() # using a translation could be better for the combination step
  count = dict() # should be relatively efficient since is a hash table

#
#  def translate(e, dic):
#    try:
#      return dic[e]
#    except KeyError:
#      dic[e] = len(dic)+1
#      return dic[e]
  def incr(e,dic):
    try:
      dic[e] += 1;
    except KeyError:
      dic[e] = 1

  chunk = list(chunk) # to avoid consuming after first pass
  ### first pass
  for bsk in chunk:
    n_bsk += 1;
    #elements = [translate(x,trsl) for x in bsk]
    for b in bsk:
      incr(b,count)

  ### filter
  to_del = []
  frequent = []
  for c in count:
    if count[c] < (n_bsk * s)/tot_bsk:
      to_del.append(c)
    else:
      frequent.append(c)
      #yield (c,1)
  for t in to_del:
    del count[t]

  """

  #frequent = list(count.keys())
  #yield (len(frequent),1)

  frequent.sort()
  cmb = list(itertools.combinations(frequent,2)) #It's probably more efficient to
  #yield (len(cmb),1)

  count_2 = dict()
  ### second pass
  for bsk in chunk:
    for pair in cmb: #this already solve the ordering problem if the list is ordered
      if pair[0] in bsk and pair[1] in bsk:
        incr(tuple(pair), count_2)

  to_del = []
  frequent = []
  for c in count_2:
    if count_2[c] < (s * n_bsk)/tot_bsk:
      to_del.append(c)
    else:
      frequent.append(c)
      yield(c,1)
  for t in to_del:
    del count_2[t]
  """
  k = 2
  while True:#len(frequent) > 0:
    #print(frequent)
    if len(frequent) == 0:
      break
    frequent.sort()
    #cmb = list(itertools.combinations(frequent, 3))
    count = dict()

    for bsk in tqdm(chunk):
      #print('basket')
      #bsk = [x for x in bsk if x in frequent] #filter only members of Ck
      filtered = set()
      for b in bsk:
        for f in frequent:
          if b in f:
            filtered.add(b)
            break;
      bsk = list(filtered)
      #print('bsk:'+str(bsk))
      bsk.sort()
      for tpl in itertools.combinations(bsk, k):
          incr(tuple(tpl), count)

    frequent = []
    for c in count:
      if count[c] > (s * n_bsk)/tot_bsk:
        frequent.append(c)
        yield(c,count[c])
    #print(len(frequent))
    k+=1






In [99]:
for a in apriori(test_basket,3,len(test_basket)):
  print(a)


100%|██████████| 11/11 [00:00<00:00, 40721.40it/s]


(('a', 'b'), 5)
(('a', 'c'), 5)
(('a', 'd'), 4)
(('a', 'e'), 4)
(('a', 'f'), 4)
(('b', 'c'), 5)
(('b', 'd'), 4)
(('b', 'e'), 4)
(('b', 'f'), 4)
(('c', 'd'), 4)
(('c', 'e'), 4)
(('c', 'f'), 4)
(('d', 'e'), 4)
(('d', 'f'), 4)
(('e', 'f'), 4)
(('f', 'q'), 4)
(('n', 'o'), 4)



100%|██████████| 11/11 [00:00<00:00, 27139.61it/s]


(('a', 'b', 'c'), 5)
(('a', 'b', 'd'), 4)
(('a', 'b', 'e'), 4)
(('a', 'b', 'f'), 4)
(('a', 'c', 'd'), 4)
(('a', 'c', 'e'), 4)
(('a', 'c', 'f'), 4)
(('a', 'd', 'e'), 4)
(('a', 'd', 'f'), 4)
(('a', 'e', 'f'), 4)
(('b', 'c', 'd'), 4)
(('b', 'c', 'e'), 4)
(('b', 'c', 'f'), 4)
(('b', 'd', 'e'), 4)
(('b', 'd', 'f'), 4)
(('b', 'e', 'f'), 4)
(('c', 'd', 'e'), 4)
(('c', 'd', 'f'), 4)
(('c', 'e', 'f'), 4)
(('d', 'e', 'f'), 4)



100%|██████████| 11/11 [00:00<00:00, 37755.60it/s]


...


100%|██████████| 11/11 [00:00<00:00, 43199.76it/s]


(('a', 'b', 'c', 'd', 'e'), 4)
(('a', 'b', 'c', 'd', 'f'), 4)
(('a', 'b', 'c', 'e', 'f'), 4)
(('a', 'b', 'd', 'e', 'f'), 4)
(('a', 'c', 'd', 'e', 'f'), 4)
(('b', 'c', 'd', 'e', 'f'), 4)



100%|██████████| 11/11 [00:00<00:00, 575.51it/s]


(('a', 'b', 'c', 'd', 'e', 'f'), 4)



100%|██████████| 11/11 [00:00<00:00, 58848.65it/s]


In [58]:

def first_red(key,value):
  yield key
def second_map(candidate, chunk):
  yield (1,1)
def second_red(key,value):
  yield key

chunk = 1; #TMP to remove
s =  4; #support threshold (maybe a fraction is better)
tot = len(baskets)

#baskets_RDD
test_RDD.mapPartitions(lambda x: apriori(x,s,tot)).reduceByKey(lambda a,b: a).collect()


KeyboardInterrupt: 

In [ ]:
print(min(actors["id"]))
print(max(actors["id"]))
print(len(actors))
len(set(actors["name"]))

In [59]:
print(list(test_basket))

[['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'l'], ['a', 'b', 'c', 'd', 'e', 'f'], ['a', 'b', 'c', 'd', 'e', 'f', 'q', 'p', 'o', 'n'], ['a', 'b', 'c', 'd', 'e', 'f', 'r', 's', 't', 'u'], ['a', 'b', 'c'], ['f', 'q'], ['f', 'q', 'z'], ['f', 'u', 'q'], ['n', 'o'], ['o', 'n'], ['n', 'o', 'i']]


In [ ]:
len(actors[actors['name'] == 'Bess Flowers'])

In [ ]:
#x = [1,2,3,4,5,6,9]
x = ["abaco","banana", "zorro","abba","abecedario"]
for i in itertools.combinations(x,2):
  print(i)

In [2]:
from tqdm import tqdm
import time

for _ in tqdm(range(1,100)):
  time.sleep(2)

100%|██████████| 99/99 [03:18<00:00,  2.00s/it]


In [49]:
[x for x in baskets if 'Mammootty' in x and 'Thilakan' in x] #(('Mammootty', 'Thilakan'), 1),

[['Ashokan', 'Thilakan', 'T G Ravi', 'Saritha', 'Mammootty', 'Nedumudi Venu'],
 ['Jagathy Sreekumar',
  'Kothuku Nanappan',
  'K. James',
  'Dominic Alummoodan',
  'Achankunju',
  'Manavalan Joseph',
  'Thilakan',
  'Ramachandran V',
  'V K Sreeraman',
  'Gopi',
  'Jayabharathi',
  'Mammootty'],
 ['Prem Prakash',
  'Sukumari',
  'Innocent',
  'Thilakan',
  'Kunchan',
  'Ratheesh',
  'Shobana',
  'Seema',
  'Mammootty',
  'Janardhanan'],
 ['Lissy',
  'Adoor Bhavani',
  'Raveendran',
  'Jose',
  'Jagathy Sreekumar',
  'Meena',
  'Sankaradi',
  'Kunchan',
  'Sukumari',
  'Thilakan',
  'Shobana',
  'Rahman',
  'Archana',
  'Mammootty',
  'Lalu Alex'],
 ['Mammootty',
  'Jayasudha',
  'Thilakan',
  'Narendra Prasad',
  'Rekha',
  'Mala Aravindan',
  'Geetha Vijayan',
  'Ashokan',
  'Shubha',
  'V K Sreeraman']]